# Philadelphia Crime Data Project
#### William McKee
#### September 2017

In this investigation, I am analyzing Philadelphia Crime Statistics from 2006 through 2015.  The data set is available on Kaggle.  I would like to know the following about crime in my home city:

(1) What parts of the city have the most crime?

(2) What crimes are most frequently committed?

(3) Has crime improved over time?

I will look at combinations to see how crime varies geographically in Philadelphia.  Perhaps thefts and murders are committed more frequently in different parts of the city.

## Data Set Basic Statistics

The investigation begins by gathering basic data about Philadelphia crime statistics.  Below, we can see the number of entries in the data set, which is slightly above 2.2 million.

In [1]:
# Initial library declarations
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Store Philadelphia's crime data
crime_data = pd.read_csv('crime.csv')

# How large is data set?
print("Data dimensions: ")
print(crime_data.shape)

Data dimensions: 
(2237605, 14)


Here, we are able to see the names of our 14 columns, and get some basic information about them.

In [2]:
# Explore column names, max values, and min values
print("Column values:")
print(crime_data.columns.values)
print("\n")

print("Column max:")
print(crime_data.max())
print("\n")

print("Column min:")
print(crime_data.min())

Column values:
['Dc_Dist' 'Psa' 'Dispatch_Date_Time' 'Dispatch_Date' 'Dispatch_Time'
 'Hour' 'Dc_Key' 'Location_Block' 'UCR_General' 'Text_General_Code'
 'Police_Districts' 'Month' 'Lon' 'Lat']


Column max:
Dc_Dist                                92
Psa                                     Z
Dispatch_Date_Time    2017-03-23 01:29:00
Dispatch_Date                  2017-03-23
Dispatch_Time                    23:59:00
Hour                                   23
Dc_Key                       201777001445
Location_Block              `732 SIGEL ST
UCR_General                          2600
Police_Districts                       22
Month                             2017-03
Lon                              -74.9575
Lat                               40.1379
dtype: object


Column min:
Dc_Dist                                       1
Psa                                           1
Dispatch_Date_Time          2006-01-01 00:00:00
Dispatch_Date                        2006-01-01
Dispatch_Time             

Here, some data columns are explored, such as police districts, crime codes, and crime descriptions.

In [3]:
# Look at some distinct data
dc_dist_distinct = crime_data.groupby('Dc_Dist')['Dc_Dist'].count()
print(dc_dist_distinct)
print("\n")

ucr_text_distinct = crime_data.groupby(['UCR_General', 'Text_General_Code']).size()
print(ucr_text_distinct)

Dc_Dist
1      48815
2     116500
3      85816
4      29198
5      31433
6      96568
7      44687
8      73820
9      84046
12    132145
14    120931
15    184677
16     73052
17     74900
18    109746
19    138987
22    127332
23     27278
24    161909
25    151245
26     86982
35    131037
39     97061
77      7813
92      1627
Name: Dc_Dist, dtype: int64


UCR_General  Text_General_Code                      
100.0        Homicide - Criminal                          3442
             Homicide - Gross Negligence                    12
             Homicide - Justifiable                         42
200.0        Rape                                        11852
300.0        Robbery Firearm                             40577
             Robbery No Firearm                          51919
400.0        Aggravated Assault Firearm                  27934
             Aggravated Assault No Firearm               68989
500.0        Burglary Non-Residential                    23276
             Burg

Finally, I seen that the date and time entries generally had certain patterns.  This code will tell us if there is any data that does not have the correct pattern which would need cleaning.  Good news - there are no incorrectly formatted date and time entries.

In [4]:
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

# Check dates and times for consistent formatting
def does_column_match_pattern(data_frame, column_name, pattern):
    ''' Returns true if every value of a data frame column matches a pattern
        data_frame = the pandas data frame to be checked
        column_name = name of column to be checked
        pattern = regular expression to be matched
    '''
    data = data_frame[column_name]
    return data.str.contains(pattern).all()

# Patterns copied from http://regexlib.com/DisplayPatterns.aspx?cattabindex=4&categoryId=5&AspxAutoDetectCookieSupport=1
date_time_re = '20\d{2}-((0[1-9])|(1[0-2]))-((0[1-9])|([1-2][0-9])|(3[0-1]))(\s)(([0-1][0-9])|(2[0-3])):([0-5][0-9]):([0-5][0-9])'
date_re = '20\d{2}-((0[1-9])|(1[0-2]))-((0[1-9])|([1-2][0-9])|(3[0-1]))'
time_re = '^(([0-1]?[0-9])|([2][0-3])):([0-5]?[0-9])(:([0-5]?[0-9]))?$'
print("Do all elements in columns match expected pattern?")
print("Dispatch_Date_Time: ", does_column_match_pattern(crime_data, 'Dispatch_Date_Time', date_time_re))
print("Dispatch_Date: ", does_column_match_pattern(crime_data, 'Dispatch_Date', date_re))
print("Dispatch_Time: ", does_column_match_pattern(crime_data, 'Dispatch_Time', time_re))
print("Hour: ", ((crime_data['Hour'] >= 0) & (crime_data['Hour'] <= 23)).all())
print("Month: ", does_column_match_pattern(crime_data, 'Month', '[0-9]{4}-[0-9]{2}'))

Do all elements in columns match expected pattern?
Dispatch_Date_Time:  True
Dispatch_Date:  True
Dispatch_Time:  True
Hour:  True
Month:  True


## Data Set Cleaning

The Philadelphia Crime Data set contains over 2.2 million rows where each row represents one 911 call about a criminal incident in the city.  Each row includes information about the date, time, police district, street, and type of incident.  There are some 2016 and 2017 incidents in the data set but there are considerable gaps in the data for those two years.  There are no apparent gaps for 2006 through 2015 so I will include only those years in the investigation.  Following this cleaning step, the data set now contains slightly over two million rows.

In [5]:
# Remove incidents from 2016 and 2017 since there are large gaps in the data
years_included = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
crime_data = crime_data[crime_data['Dispatch_Date_Time'].astype(str).str.startswith(tuple(years_included))]

# How large is data set now?
print("Data dimensions: ")
print(crime_data.shape)

Data dimensions: 
(2035062, 14)


There are less than 1000 entries which do not contain a crime classification and corresponding description.  Since I wish to measure the crime type, I will remove these rows from the data set.

In [6]:
# Exclude rows without a crime type or description
crime_data.dropna(subset=['UCR_General', 'Text_General_Code'], inplace = True)

# How large is data set now?
print("Data dimensions: ")
print(crime_data.shape)

Data dimensions: 
(2035062, 14)


When I looked at the distinct police districts represented in the dataset, I compared the district numbers to a map of the Philadelphia Police Districts.  There are four districts which no longer exist due to mergers.  The 4th district was merged into the 3rd district.  The 23rd was absored by the 22nd district.  The 77th was absorbed by the 12th district.  Meanwhile, the 92nd district is now part of the 14th district.  The same merger will occur in my data frame.

In [7]:
# Check before merger
print("Before district merger:")
dc_dist_distinct = crime_data.groupby('Dc_Dist')['Dc_Dist'].count()
print(dc_dist_distinct)
print("\n")

# Merge districts
crime_data.loc[crime_data.Dc_Dist == 4, 'Dc_Dist'] = 3
crime_data.loc[crime_data.Dc_Dist == 23, 'Dc_Dist'] = 22
crime_data.loc[crime_data.Dc_Dist == 77, 'Dc_Dist'] = 12
crime_data.loc[crime_data.Dc_Dist == 92, 'Dc_Dist'] = 14

# Check the result of the merger
print("After district merger:")
dc_dist_distinct = crime_data.groupby('Dc_Dist')['Dc_Dist'].count()
print(dc_dist_distinct)
print("\n")

Before district merger:
Dc_Dist
1      44775
2     106155
3      76989
4      29197
5      28582
6      87384
7      40653
8      67759
9      76028
12    121702
14    109950
15    167387
16     66705
17     69817
18     99031
19    124281
22    113371
23     27278
24    146134
25    138099
26     78988
35    119559
39     86675
77      6936
92      1627
Name: Dc_Dist, dtype: int64


After district merger:
Dc_Dist
1      44775
2     106155
3     106186
5      28582
6      87384
7      40653
8      67759
9      76028
12    128638
14    111577
15    167387
16     66705
17     69817
18     99031
19    124281
22    140649
24    146134
25    138099
26     78988
35    119559
39     86675
Name: Dc_Dist, dtype: int64




The **Location_Block** field is not consistent so there will be some cleaning needed to ensure the same pattern in the data.  The cleanup code will ensure that the locations are specified as either a block or an intersection.  A block is denoted as _2400 BLOCK FRONT STREET_.  An intersection fits the pattern _RHAWN ST / COTTMAN AV_.  The street types are standarized to ensure consistency as much as practical.  For example, all _ROAD_ street types are changed to _RD_.  Rows with empty or invalid locations (such as a single word) are removed from the data set.

In [8]:
import re

# Regular expressions
block_re = '^(\d+)(\s)BLOCK(\s)(.+)(\s)(\w+)$'
block_missed_re = '^(\d+)(\s)(.+)(\s)(\w+)$'
block_street_missed_re = '^(\d+)(.+)$'
street_missed_re = '^(\d+)(\s)BLOCK(\s)(.+)$'
int_amp_re = '^(.+)(\s)(\w+)(\s)&(.+)(\s)(\w+)$'
int_slash_re = '^(.+)(\s)(\w+)(\s)(\/)(.+)(\s)(\w+)$'

# Location formatting function
def format_location_data(item):
    ''' Format data set's location block column (invoked via apply method)
        item = one item from location block column
    '''  
    # Initial trimming
    item = item.strip()
    
    # Remove extraneous first characters
    if (item[0] == '`' or item[0] == '/'):
        item = item[1:]
        
    # Remove item with no whitespace
    if (len(item.split()) == 1):
        item = 'None'
    
    # We will not look at establishment, only the street
    if ('@' in item):
        at_index = item.index('@')
        item = item[at_index+1:] # also remove '@'
        item = item.strip()

    # BLOCK and SLASH patterns accepted at this stage
    if (re.fullmatch(block_re, item) or re.fullmatch(int_slash_re, item)):
        pass
    
    elif (re.fullmatch(street_missed_re, item)):
        # Add default street type
        item += " ST"
    
    elif (re.fullmatch(block_missed_re, item)):
        # Add BLOCK after number
        tokens = item.split()
        item = tokens[0] + " BLOCK"
        
        # Reconstruct string
        for i in range(1,len(tokens)):
            item = item + " " + tokens[i]
    
    elif (re.fullmatch(block_street_missed_re, item)):
        # Add BLOCK after number
        tokens = item.split()
        item = tokens[0] + " BLOCK"
        
        # Reconstruct string
        for i in range(1,len(tokens)):
            item = item + " " + tokens[i]
            
        # Add default street type
        item += " ST"
    
    elif (re.fullmatch(int_amp_re, item)):
        # Replace ampersand
        item = item.replace("&", "/")
    
    return item

# Street Types
STREET_TYPE_EXPECTED = ["AV", "BLVD", "CIR", "CT", "DR", "LN", "PKWY", "PL", "RD", "ROW", "ST", "TER", "WAY"]

STREET_TYPE_MAPPING = { "AVE": "AV",
                        "AVENUE": "AV",
                        "AVE": "AV",
                        "BLD": "BLVD",
                        "BLV": "BLVD",
                        "BDV": "BLVD",
                        "BOULEVARD": "BLVD",
                        "CI": "CIR",
                        "CIRCLE": "CIR",
                        "CRT": "CT",
                        "COURT": "CT",
                        "DRIVE": "DR",
                        "LANE": "LN",
                        "PKY": "PKWY",
                        "PWY": "PKWY",
                        "PARKWAY": "PKWY",
                        "PLA": "PL",
                        "PLACE": "PL",
                        "RDS": "RD",
                        "ROAD": "RD",
                        "STR": "ST",
                        "STT": "ST",
                        "STREET": "ST",
                        "TRCE": "TER",
                        "WA": "WAY"
                      }

# Street Type Formatting function
def format_street_type(item):
    ''' Format street type in data set's location block column (invoked via apply method)
        This function is invoked agter format_location_data
        item = one item from location block column
    '''
    # BLOCK pattern
    if (re.fullmatch(block_re, item)):
        tokens = item.split()
    
        # Check the hundred block
        if (tokens[0].isdigit()):
            # Check the hundred block
            tokens = item.split()
            block_num = int(tokens[0])
            block_num = (block_num // 100) * 100
            item = str(block_num)
        else:
            item = tokens[0]
            
        # Keep BLOCK and name of street
        item = item + " " + tokens[1] + " " + tokens[2]
    
        # Check remaining tokens for street type
        for i in range(3, len(tokens)):
            if (tokens[i] in STREET_TYPE_MAPPING):
                item = item + " " + STREET_TYPE_MAPPING[tokens[i]]
                break
            elif (tokens[i] in STREET_TYPE_EXPECTED):
                item = item + " " + tokens[i]
                break
            else:
                item = item + " " + tokens[i]
    
    # SLASH pattern
    elif (re.fullmatch(int_slash_re, item)):
        street_names = item.split('/')
        
        # Go through each part
        this_part = ""
        for i in range(0, len(street_names)):
            if (i>0):
                # Previous part
                item = this_part + " / "
                this_part = ""
                
            # Initial trimming
            street_names[i].strip()
            
            # Split this part
            tokens = street_names[i].split()
            
            # Empty item?
            if (len(tokens) == 0):
                break
        
            # Keep first token (for case like STREET RD)
            this_part = this_part + tokens[0]
            
            # Check remaining tokens for street type
            for j in range(1, len(tokens)):
                if (tokens[j] in STREET_TYPE_MAPPING):
                    this_part = this_part + " " + STREET_TYPE_MAPPING[tokens[j]]
                    break
                elif (tokens[j] in STREET_TYPE_EXPECTED):
                    this_part = this_part + " " + tokens[j]
                    break
                else:
                    this_part = this_part + " " + tokens[j]
            
        # Last part
        item = item + this_part

    return item

# Standardize the location data
crime_data['Location_Block'] = crime_data['Location_Block'].apply(format_location_data)

# Dump items with no information
crime_data = crime_data[crime_data['Location_Block'] != 'None']

# Standardize street type
crime_data['Location_Block'] = crime_data['Location_Block'].apply(format_street_type)

# How large is data set now?
print("Data dimensions: ")
print(crime_data.shape)

Data dimensions: 
(2034110, 14)


There is one more cleaning step that must take place.  If multiple people placed a 911 call for the same incident, the incident is recorded twice in this data set.  These rows must be combined so that each incident is counted only once regardless of the number of 911 callers.  After removing such duplicates, the number of rows in the data set is about 1.95 million, or 80000 fewer than the previous iteration.

In [9]:
# Try groupby to remove duplicates
crime_data = crime_data[['Dispatch_Date_Time', 'Location_Block', 'UCR_General', 'Text_General_Code']].drop_duplicates()

# Sort data set to better see repeat incidents
crime_data.sort_values(['Dispatch_Date_Time'], ascending=False, inplace=True)

# How large is data set now?
print("Data dimensions: ")
print(crime_data.shape)

Data dimensions: 
(1953458, 4)


#### TODO: Data cleaning is finished; plot and analyze data